In [365]:
import tensorflow as tf
import tensorflow.keras as keras
import os
import sys
from importlib import reload
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import probdownscale
reload(probdownscale.TaskExtractor)
reload(probdownscale.MetaTrain)
from probdownscale.MetaTrain import MetaSGD

from probdownscale.TaskExtractor import TaskExtractor

import numpy as np
import netCDF4 as nc

## 1. Debug TaskExtractor

In [53]:
file_path_g_05 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20050516-20060515.nc'
file_path_g_06 =  r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20060516-20070515.nc'
file_path_m = r'C:\Users\96349\Documents\Downscale_data\MERRA2\MERRA2_aerosol_variables_over_MiddleEast_daily_20000516-20180515.nc'
target_var = 'BCSMASS'

# read data
g05_data = nc.Dataset(file_path_g_05)
g06_data = nc.Dataset(file_path_g_06)
m_data_nc = nc.Dataset(file_path_m)

# define lat&lon of MERRA, G5NR and mete
M_lons = m_data_nc.variables['lon'][:]
# self.M_lons = (M_lons-M_lons.mean())/M_lons.std()
M_lats = m_data_nc.variables['lat'][:]
# self.M_lats = (M_lats-M_lats.mean())/M_lats.std()
G_lons = g05_data.variables['lon'][:]
# self.G_lons = (G_lons-G_lons.mean())/G_lons.std()
G_lats = g05_data.variables['lat'][:]

# extract target data
g_data = np.concatenate((g05_data.variables[target_var][:], g06_data.variables[target_var][:]), axis=0)
m_data = m_data_nc.variables[target_var][5*365:7*365]

C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packag

In [339]:
data = [g_data, m_data]
lats_lons = [G_lats, G_lons, M_lats, M_lons]
task_dim = 10
test_proportion = 0.3
n_lag = 10
taskextractor = TaskExtractor(data, lats_lons, task_dim, test_proportion, n_lag)

In [333]:
train_x, train_y, test_x, test_y = taskextractor.get_random_tasks(10)


In [340]:
train_x, train_y, test_x, test_y = taskextractor._get_random_task(is_random=False, record=False, lat_lon=(13.5625, 49.375))

In [342]:
len(train_x[0])

504

## 2. Test Network

In [145]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow_probability import distributions as tfd

In [243]:
# the last channel is the comonents
alpha = np.random.rand(3,2,5)
alpha = alpha/alpha.sum()
mu = np.ones((3, 2, 5))
test_md = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Exponential(
        rate=mu))

In [279]:
test_md.log_prob(np.ones((3,2)))

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[-1., -1.],
       [-1., -1.],
       [-1., -1.]])>

In [160]:
a =  np.random.rand(3,3,3)
#a = (a - a.min())/(a.max() - a.min())
a = a/a.sum()
a.sum()

1.0

In [322]:
def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

components = 100
no_parameters = 2
def slice_parameter_vectors(parameter_vector):
    return [parameter_vector[:, i * task_dim*task_dim*components:(i + 1) *task_dim*task_dim*components] for i in range(no_parameters)]

def gnll_loss(y, parameter_vector):
    alpha, mu = slice_parameter_vectors(parameter_vector)  # Unpack parameter vectors
    #print(alpha.shape, mu.shape)
    alpha1 = tf.reshape(alpha, (tf.shape(alpha)[0], task_dim, task_dim, components))
    mu1 = tf.reshape(mu, (tf.shape(mu)[0], task_dim, task_dim, components))
    #print(alpha1.shape, mu1.shape)
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha1),
        components_distribution=tfd.Exponential(
        rate=mu1), allow_nan_stats=False
    )

    log_likelihood = gm.log_prob(y)  # Evaluate log-probability of y
    
    return -tf.reduce_mean(log_likelihood, axis=-1)

tf.keras.utils.get_custom_objects().update({'nnelu': layers.Activation(nnelu)})

In [328]:
# MDN model
# input dim (time, channel, rows, cols)
input1 = layers.Input(shape=(n_lag, 1, task_dim, task_dim)) 
input2 = layers.Input(shape=(task_dim, task_dim, 1))
input3 = layers.Input(shape=(1))

X = layers.ConvLSTM2D(filters=20, kernel_size=(1,2), activation='tanh', return_sequences=False)(input1)
X = layers.Flatten()(X)

X1 = layers.Conv2D(20, (2,2), activation='tanh')(input2)
X1 = layers.Flatten()(X1)
X2 = layers.BatchNormalization()(input3)
X2 = layers.Dense(30, activation='relu')(X2)

X = layers.Concatenate()([X, X1, X2])

alphas = layers.Dense(components*task_dim*task_dim, activation="softmax")(X)
#alphas = layers.Reshape((task_dim, task_dim, components), name="alphas")(alphas)
mus = layers.Dense(components*task_dim*task_dim, activation="nnelu")(X)
#mus = layers.Reshape((task_dim, task_dim, components) ,name="mus")(mus)
#sigmas = layers.Dense(components, activation="nnelu", name="sigmas")(X)
output = layers.Concatenate()([alphas, mus])
MDN_model = Model([input1, input2, input3], output)
MDN_model.compile(optimizer='adam', loss=gnll_loss)
MDN_model.summary()

Model: "model_30"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_68 (InputLayer)          [(None, 10, 1, 10,   0           []                               
                                10)]                                                              
                                                                                                  
 input_69 (InputLayer)          [(None, 10, 10, 1)]  0           []                               
                                                                                                  
 input_70 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 conv_lstm2d_34 (ConvLSTM2D)    (None, 1, 9, 20)     4880        ['input_68[0][0]']        

In [329]:
MDN_model.fit([train_x_1, train_x_2, train_x_3], train_y, epochs=10)

Epoch 1/10
16/16 [==============================] - 7s 260ms/step - loss: -0.0502
Epoch 2/10
16/16 [==============================] - 4s 266ms/step - loss: -0.4720
Epoch 3/10
16/16 [==============================] - 4s 269ms/step - loss: -1.4893
Epoch 4/10
16/16 [==============================] - 4s 262ms/step - loss: -2.5068
Epoch 5/10
16/16 [==============================] - 4s 261ms/step - loss: -3.1417
Epoch 6/10
16/16 [==============================] - 4s 270ms/step - loss: -3.5134
Epoch 7/10
16/16 [==============================] - 4s 262ms/step - loss: -3.7662
Epoch 8/10
16/16 [==============================] - 4s 262ms/step - loss: -3.9584
Epoch 9/10
16/16 [==============================] - 4s 261ms/step - loss: -4.1177
Epoch 10/10
16/16 [==============================] - 4s 261ms/step - loss: -4.2559


## 3. Debug Meta Learning Algorithm

In [346]:
# define necessary tool functions
components = 100
no_parameters = 2

def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

def slice_parameter_vectors(parameter_vector):
    return [parameter_vector[:, i * task_dim*task_dim*components:(i + 1) *task_dim*task_dim*components] for i in range(no_parameters)]

def gnll_loss(y, parameter_vector):
    alpha, mu = slice_parameter_vectors(parameter_vector)  # Unpack parameter vectors
    #print(alpha.shape, mu.shape)
    alpha1 = tf.reshape(alpha, (tf.shape(alpha)[0], task_dim, task_dim, components))
    mu1 = tf.reshape(mu, (tf.shape(mu)[0], task_dim, task_dim, components))
    #print(alpha1.shape, mu1.shape)
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha1),
        components_distribution=tfd.Exponential(
        rate=mu1), allow_nan_stats=False
    )

    log_likelihood = gm.log_prob(y)  # Evaluate log-probability of y
    
    return -tf.reduce_mean(log_likelihood, axis=-1)

tf.keras.utils.get_custom_objects().update({'nnelu': layers.Activation(nnelu)})

# define MDN model
# input dim (time, channel, rows, cols)
input1 = layers.Input(shape=(n_lag, 1, task_dim, task_dim)) 
input2 = layers.Input(shape=(task_dim, task_dim, 1))
input3 = layers.Input(shape=(1))

X = layers.ConvLSTM2D(filters=20, kernel_size=(1,2), activation='tanh', return_sequences=False)(input1)
X = layers.Flatten()(X)

X1 = layers.Conv2D(20, (2,2), activation='tanh')(input2)
X1 = layers.Flatten()(X1)
X2 = layers.BatchNormalization()(input3)
X2 = layers.Dense(30, activation='relu')(X2)

X = layers.Concatenate()([X, X1, X2])

alphas = layers.Dense(components*task_dim*task_dim, activation="softmax")(X)
#alphas = layers.Reshape((task_dim, task_dim, components), name="alphas")(alphas)
mus = layers.Dense(components*task_dim*task_dim, activation="nnelu")(X)
#mus = layers.Reshape((task_dim, task_dim, components) ,name="mus")(mus)
#sigmas = layers.Dense(components, activation="nnelu", name="sigmas")(X)
output = layers.Concatenate()([alphas, mus])
MDN_model = Model([input1, input2, input3], output)

# define TaskExtractor
data = [g_data, m_data]
lats_lons = [G_lats, G_lons, M_lats, M_lons]
task_dim = 10
test_proportion = 0.3
n_lag = 10
taskextractor = TaskExtractor(data, lats_lons, task_dim, test_proportion, n_lag)

In [402]:
reload(probdownscale.MetaTrain)
from probdownscale.MetaTrain import MetaSGD
# define meta learner
meta_step = 10
meta_optimizer = tf.keras.optimizers.RMSprop(0.1)
inner_step = 1
inner_optimizer = tf.keras.optimizers.SGD()

meta_learner = MetaSGD(MDN_model, gnll_loss, meta_step, meta_optimizer, inner_step, inner_optimizer, taskextractor)

In [406]:
meta_learner._train_on_batch(10)

origional lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0123>
updated lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0123>


<tf.Tensor: shape=(), dtype=float32, numpy=-4.17285>

In [379]:
for t in MDN_model.get_weights():
    print(t.shape)
    print(type(t))

(1, 2, 10, 80)
<class 'numpy.ndarray'>
(1, 2, 20, 80)
<class 'numpy.ndarray'>
(80,)
<class 'numpy.ndarray'>
(2, 2, 1, 20)
<class 'numpy.ndarray'>
(20,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1, 30)
<class 'numpy.ndarray'>
(30,)
<class 'numpy.ndarray'>
(1830, 10000)
<class 'numpy.ndarray'>
(10000,)
<class 'numpy.ndarray'>
(1830, 10000)
<class 'numpy.ndarray'>
(10000,)
<class 'numpy.ndarray'>


In [378]:
for t in MDN_model.trainable_variables:
    print(t.shape)

(1, 2, 10, 80)
(1, 2, 20, 80)
(80,)
(2, 2, 1, 20)
(20,)
(1,)
(1,)
(1, 30)
(30,)
(1830, 10000)
(10000,)
(1830, 10000)
(10000,)


In [382]:
np.random.normal(1, 1, (10,10))

array([[-0.42265273,  0.06462992,  0.78516736,  1.25093588,  0.83372054,
         0.12684125,  1.23409446,  2.26263361,  1.56729192,  2.31819973],
       [ 1.93449083,  1.08668188,  1.85857859, -0.48187132,  0.83773894,
        -0.0148779 ,  0.18204529,  1.47166016,  0.96289486,  1.22284512],
       [ 0.30987924,  0.51762376,  1.64761627,  0.68419885,  1.3715202 ,
         1.5960097 ,  0.38286023,  1.53599298,  0.92622567,  2.26660407],
       [ 0.45589063,  1.61386188,  0.73829364,  2.3284075 ,  0.28444655,
         0.69278491,  1.0866214 ,  1.92874451,  1.35150042,  2.27992891],
       [ 1.9689489 ,  0.62789535,  1.60048962,  0.38896286,  1.93708888,
         2.4220651 ,  2.02954596,  1.27148274,  0.48648999,  1.17837296],
       [ 1.39271132,  2.66557933,  0.47243303,  1.23612533,  0.91220452,
         0.77176904, -1.50397331,  2.02389278,  2.83962546,  1.29593856],
       [ 1.72467709,  1.00786031,  1.8163368 ,  1.8785098 ,  0.48277863,
         2.45905549,  0.27121147,  1.10766889